---
title: Data extraction
description: We proceed to extract dataset using WorldFootballR library from Fbref and ...
---

We collect data from Fbref and Transfermarkt using the WorldFootballR library. 

We collect data from 2015 to 2023 from major european first division leagues : England, Spain, Italy, Germany, France, Portugal, Scotland, Poland, Greece, Turkey, Switzerland, Netherlands, Belgium, Austria.

In [21]:
if (!require(worldfootballR)) { 
    install.packages("worldfootballR")
    library(worldfootballR)
}

if (!require(readr)) {
  install.packages("readr")
  library(readr)
}

### Collecting match results

In [22]:
# Change parameter to study different teams and seaons 
# country <- c("ENG", "ESP", "ITA", "GER", "FRA", "POR", "SCO", "POL", "GRE", "SUI", "NED", "BEL", "AUT")
# year <- c(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023)

country <- c("ENG", "ESP", "ITA", "GER", "FRA")
year <- c(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023)
match_result <- fb_match_results(country = country, gender = "M", season_end_year = year, tier = "1st")

In [23]:
columns_to_keep <- c('Competition_Name', 'Country', 'Season_End_Year', 'Date', 'Home', 'HomeGoals', 'Away', 'AwayGoals')
match_result <- match_result[, columns_to_keep]
# Rename columns
colnames(match_result) <- c('League', 'Country', 'Season', 'Date', 'Home', 'HomeGoals', 'Away', 'AwayGoals')
head(match_result)

,League,Country,Season,Date,Home,HomeGoals,Away,AwayGoals
,<chr>,<chr>,<int>,<date>,<chr>,<dbl>,<chr>,<dbl>
1,Premier League,ENG,2015,2014-08-16,Manchester Utd,1,Swansea City,2
2,Premier League,ENG,2015,2014-08-16,Stoke City,0,Aston Villa,1
3,Premier League,ENG,2015,2014-08-16,Leicester City,2,Everton,2
4,Premier League,ENG,2015,2014-08-16,QPR,0,Hull City,1
5,Premier League,ENG,2015,2014-08-16,West Ham,0,Tottenham,1
6,Premier League,ENG,2015,2014-08-16,West Brom,2,Sunderland,2


In [24]:
unique(match_result$League)
# Fix League name for Bundesliga
match_result$League <- gsub("Fu\303\237ball-Bundesliga", "Bundesliga", match_result$League)

[1] "Premier League"            "La Liga"                  
[3] "Ligue 1"                   "Fu\303\237ball-Bundesliga"
[5] "Serie A"

In [25]:
summary(match_result)

    League            Country              Season          Date           
 Length:16463       Length:16463       Min.   :2015   Min.   :2014-08-08  
 Class :character   Class :character   1st Qu.:2017   1st Qu.:2016-10-29  
 Mode  :character   Mode  :character   Median :2019   Median :2019-01-11  
                                       Mean   :2019   Mean   :2019-01-10  
                                       3rd Qu.:2021   3rd Qu.:2021-03-21  
                                       Max.   :2023   Max.   :2023-06-11  
                                                                          
     Home             HomeGoals          Away             AwayGoals    
 Length:16463       Min.   : 0.000   Length:16463       Min.   :0.000  
 Class :character   1st Qu.: 1.000   Class :character   1st Qu.:0.000  
 Mode  :character   Median : 1.000   Mode  :character   Median :1.000  
                    Mean   : 1.536                      Mean   :1.213  
                    3rd Qu.: 2.000      

In [26]:
# Saving the data
write_csv(match_result, "data/extracted_match_results.csv")

### Collecting head coach data

In [27]:
# Change country to study different teams
# country <- c("England", "Spain", "Italy", "Germany", "France", "Portugal", "Scotland", "Poland", "Greece", "Switzerland", "Netherlands", "Belgium")

country <- c("England", "Spain", "Italy", "Germany", "France")
teams_url <- c()

for (i in seq_along(country)) {
    league_team_url <- tm_league_team_urls(country_name = country[i], start_year = 2015)
    teams_url <- c(teams_url, league_team_url)
}

In [28]:
head_coach <- tm_team_staff_history(team_urls = teams_url, staff_role = "Manager")
unique(head_coach$league)
unique(head_coach$team)

[1] "Premier League"                "Championship"                 
 [3] "LaLiga"                        "Primera Federaci<U+00F3>n - Grupo II"
 [5] "LaLiga2"                       "Primera Federaci<U+00F3>n - Grupo I"
 [7] "Serie A"                       "Serie B"                      
 [9] "Serie D - Girone D"            NA                             
[11] "Bundesliga"                    "2. Bundesliga"                
[13] "3. Liga"                       "Ligue 1"                      
[15] "Ligue 2"

[1] "Chelsea FC"               "Manchester City"         
 [3] "Arsenal FC"               "Manchester United"       
 [5] "Liverpool FC"             "Tottenham Hotspur"       
 [7] "Everton FC"               "Southampton FC"          
 [9] "West Ham United"          "Newcastle United"        
[11] "Leicester City"           "Stoke City"              
[13] "Swansea City"             "Watford FC"              
[15] "Crystal Palace"           "Aston Villa"             
[17] "Norwich City"             "West Bromwich Albion"    
[19] "Sunderland AFC"           "AFC Bournemouth"         
[21] "FC Barcelona"             "Real Madrid"             
[23] "Atl<U+00E9>tico de Madrid" "Valencia CF"             
[25] "Sevilla FC"               "Athletic Bilbao"         
[27] "Villarreal CF"            "Real Sociedad"           
[29] "Celta de Vigo"            "M<U+00E1>laga CF"        
[31] "Granada CF"               "RCD Espanyol Barcelona"  
[33] "Deportivo de La Coru<U+00F1>a" "SD Eibar"                
[35] "Real Betis Balompi<U+00E9>" "Sporting Gij<U+00F3>n"   
[37] "Levante UD"               "Rayo Vallecano"          
[39] "Getafe CF"                "UD Las Palmas"           
[41] "Juventus FC"              "AS Roma"                 
[43] "SSC Napoli"               "Inter Milan"             
[45] "AC Milan"                 "ACF Fiorentina"          
[47] "SS Lazio"                 "UC Sampdoria"            
[49] "Genoa CFC"                "US Sassuolo"             
[51] "Udinese Calcio"           "Torino FC"               
[53] "Atalanta BC"              "Bologna FC 1909"         
[55] "FC Empoli"                "Palermo FC"              
[57] "AC Carpi"                 "Chievo Verona"           
[59] "Hellas Verona"            "Frosinone Calcio"        
[61] "Bayern Munich"            "Borussia Dortmund"       
[63] "VfL Wolfsburg"            "Bayer 04 Leverkusen"     
[65] "FC Schalke 04"            "Borussia M<U+00F6>nchengladbach"
[67] "TSG 1899 Hoffenheim"      "1.FSV Mainz 05"          
[69] "VfB Stuttgart"            "Hertha BSC"              
[71] "Eintracht Frankfurt"      "FC Augsburg"             
[73] "1.FC K<U+00F6>ln"         "SV Werder Bremen"        
[75] "Hamburger SV"             "Hannover 96"             
[77] "FC Ingolstadt 04"         "SV Darmstadt 98"         
[79] "Paris Saint-Germain"      "AS Monaco"               
[81] "Olympique Lyon"           "Olympique Marseille"     
[83] "AS Saint-<U+00C9>tienne"  "Stade Rennais FC"        
[85] "FC Girondins Bordeaux"    "OGC Nice"                
[87] "LOSC Lille"               "FC Lorient"              
[89] "Montpellier HSC"          "Stade Reims"             
[91] "FC Toulouse"              "FC Nantes"               
[93] "SM Caen"                  "EA Guingamp"             
[95] "Angers SCO"               "ESTAC Troyes"            
[97] "SC Bastia"                "GFC Ajaccio"

Filter leagues that are not First Division Leagues

In [35]:
# Filter teams that are not First Division teams
# first_division_teams <- c(
#     'Premier League', 'LaLiga', 'Serie A', 'Bundesliga', 'Ligue 1', 
#     'Liga Portugal', 'Scottish Premiership', 'PKO BP Ekstraklasa', 'Super League 1', 
#     'Super League', 'Eredivisie', 'Jupiler Pro League')

first_division_teams <- c('Premier League', 'LaLiga', 'Serie A', 'Bundesliga', 'Ligue 1')
# Ensure the every first_division_teams is in the head_coach$league
all(first_division_teams %in% head_coach$league)
# Filter the head_coach data
head_coach <- head_coach[head_coach$league %in% first_division_teams, ]
head(head_coach, 5)

[1] TRUE

,team_name,league,country,staff_role,staff_name,staff_url,staff_dob,staff_nationality,staff_nationality_secondary,appointed,end_date,days_in_post,matches,wins,draws,losses,ppg
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Chelsea FC,Premier League,England,Manager,Mauricio Pochettino,https://www.transfermarkt.com/mauricio-pochettino/profil/trainer/9044,"Mar 2, 1972",Argentina,Spain,2023-07-01,NA,284,42,21,9,12,1.71
2,Chelsea FC,Premier League,England,Manager,Graham Potter,https://www.transfermarkt.com/graham-potter/profil/trainer/23954,"May 20, 1975",England,NA,2022-09-08,2023-04-02,206,31,12,8,11,1.42
3,Chelsea FC,Premier League,England,Manager,Thomas Tuchel,https://www.transfermarkt.com/thomas-tuchel/profil/trainer/7471,"Aug 29, 1973",Germany,NA,2021-01-26,2022-09-07,589,100,63,19,18,2.08
4,Chelsea FC,Premier League,England,Manager,Frank Lampard,https://www.transfermarkt.com/frank-lampard/profil/trainer/60805,"Jun 20, 1978",England,NA,2019-07-04,2021-01-25,571,84,44,15,25,1.75
5,Chelsea FC,Premier League,England,Manager,Maurizio Sarri,https://www.transfermarkt.com/maurizio-sarri/profil/trainer/10073,"Jan 10, 1959",Italy,NA,2018-07-14,2019-06-30,351,63,40,11,12,2.08


In [37]:
columns_to_keep <- c('team_name', 'league', 'country', 'staff_name', 'appointed', 'end_date', 'days_in_post', 'matches', 'wins', 'draws', 'losses')
head_coach <- head_coach[, columns_to_keep]

# Rename columns
colnames(head_coach) <- c('Team', 'League', 'Country', 'HeadCoach', 'Appointed', 'EndDate', 'Tenure', 'Matches', 'Wins', 'Draws', 'Losses')

summary(head_coach)

     Team              League            Country           HeadCoach        
 Length:3501        Length:3501        Length:3501        Length:3501       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
                                                                            
   Appointed             EndDate               Tenure           Matches       
 Min.   :1886-06-26   Min.   :1893-08-01   Min.   : -242.0   Min.   :   0.00  
 1st Qu.:1961-07-01   1st Qu.:1963-04-01   1st Qu.:  186.0   1st Qu.:  10.00  
 Median :1986-11-06   Median :1987-10-01   Median :  364.0   Median :  29.00  
 Mean   :1982-03-27   Mean   :1983-02-23   Mean   :  608.3   Mean   

In [38]:
# Saving the data
write_csv(head_coach, "data/extracted_head_coach.csv")